In [5]:
import cv2
import numpy as np

def detect_piano_pattern(image_path):
    """
    Detects patterns resembling piano keys in an image using Hough Transform and basic contrast analysis.

    Parameters:
        image_path (str): Path to the input image.

    Returns:
        bool: True if a piano-like pattern (4:10 black-to-white keys) is detected, False otherwise.
    """
    # Load the image and convert to grayscale
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply edge detection
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)

    # Detect lines using Hough Transform
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 250)

    if lines is None:
        print("No lines detected.")
        return False

    # Visualize all detected lines
    output_image = image.copy()
    if lines is not None:
        for line in lines:
            rho, theta = line[0]
            a, b = np.cos(theta), np.sin(theta)
            x0, y0 = a * rho, b * rho
            x1 = int(x0 + 1000 * (-b))
            y1 = int(y0 + 1000 * (a))
            x2 = int(x0 - 1000 * (-b))
            y2 = int(y0 - 1000 * (a))
            cv2.line(output_image, (x1, y1), (x2, y2), (255, 0, 0), 1)

    # Save and display the lines image
    all_lines_path = "output_with_all_lines.jpg"
    cv2.imwrite(all_lines_path, output_image)
    cv2.imshow("All Detected Lines", output_image)
    cv2.waitKey(0)

    # Extract strong lines and their parameters
    strong_lines = []
    for line in lines:
        rho, theta = line[0]
        strong_lines.append((rho, theta))

    # Sort lines by their rho value to group approximately parallel lines
    strong_lines.sort(key=lambda x: x[0])

    # Analyze contrast between adjacent lines
    highest_contrast = 0
    best_pair = None

    for i in range(len(strong_lines) - 1):
        rho1, _ = strong_lines[i]
        rho2, _ = strong_lines[i + 1]

        # Extract the region between the lines
        mask = np.zeros_like(gray)
        cv2.line(mask, (0, int(rho1)), (gray.shape[1], int(rho1)), 255, 1)
        cv2.line(mask, (0, int(rho2)), (gray.shape[1], int(rho2)), 255, 1)

        region = cv2.bitwise_and(gray, gray, mask=mask)

        # Check pixel intensity distribution in the region
        total_pixels = np.sum(region > 0)
        white_pixels = np.sum(region > 100)  # Nearly white pixels
        black_pixels = np.sum(region < 50)   # Nearly black pixels

        if total_pixels == 0:
            continue

        white_ratio = white_pixels / total_pixels
        black_ratio = black_pixels / total_pixels

        print(f"Line pair (rho1={rho1}, rho2={rho2}): White ratio = {white_ratio:.2f}, Black ratio = {black_ratio:.2f}")

        # Check if the region has approximately 65% white and 35% black pixels
        if 0.6 <= white_ratio <= 0.7 and 0.3 <= black_ratio <= 0.4:
            contrast = np.std(region[region > 0])

            # Update the highest contrast and best line pair
            if contrast > highest_contrast:
                highest_contrast = contrast
                best_pair = (rho1, rho2)

    # Visualize the detected lines and best region
    output_image = image.copy()
    if best_pair:
        rho1, rho2 = best_pair
        cv2.line(output_image, (0, int(rho1)), (output_image.shape[1], int(rho1)), (0, 255, 0), 2)
        cv2.line(output_image, (0, int(rho2)), (output_image.shape[1], int(rho2)), (0, 255, 0), 2)

    # Save and display the output image
    output_path = "output_with_detected_pattern.jpg"
    cv2.imwrite(output_path, output_image)
    cv2.imshow("Detected Pattern", output_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Check if the highest contrast pair meets the threshold
    if best_pair:
        print(f"Potential piano-like pattern detected between lines at rho={best_pair[0]} and rho={best_pair[1]} with contrast={highest_contrast}.")
        return True

    print("No piano-like pattern detected.")
    return False

# Example usage
image_path = "/Users/tunaonat/Desktop/proje-git/piano-transcription/xd/example.png"
detect_piano_pattern(image_path)


Line pair (rho1=8.0, rho2=34.0): White ratio = 0.12, Black ratio = 181.81
Line pair (rho1=34.0, rho2=37.0): White ratio = 0.36, Black ratio = 181.89
Line pair (rho1=37.0, rho2=175.0): White ratio = 0.51, Black ratio = 182.25
Line pair (rho1=175.0, rho2=194.0): White ratio = 0.52, Black ratio = 182.13
Line pair (rho1=194.0, rho2=196.0): White ratio = 0.51, Black ratio = 181.93
Line pair (rho1=196.0, rho2=201.0): White ratio = 0.51, Black ratio = 181.81
Line pair (rho1=201.0, rho2=204.0): White ratio = 0.53, Black ratio = 181.59
Line pair (rho1=204.0, rho2=206.0): White ratio = 0.64, Black ratio = 181.32
Line pair (rho1=206.0, rho2=208.0): White ratio = 0.77, Black ratio = 181.20
Line pair (rho1=208.0, rho2=212.0): White ratio = 0.85, Black ratio = 181.15
Line pair (rho1=212.0, rho2=264.0): White ratio = 0.89, Black ratio = 181.03
Line pair (rho1=264.0, rho2=267.0): White ratio = 0.88, Black ratio = 181.03
Line pair (rho1=267.0, rho2=269.0): White ratio = 0.89, Black ratio = 181.03
Line 

False